In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import konlpy

data = pd.read_csv("data/sample_text.csv", header=None, names=['text'])
data.head()

,text
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...
1,하나의 인프라 기술이기도 하다
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다


In [2]:
tokenizer = konlpy.tag.Mecab()

data['text'] = data['text'].str.replace(",", "")
data['morphs'] = data['text'].map(lambda x: tokenizer.morphs(x))
data

,text,morphs
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...,"[인공지능, 또는, AI, 는, 인간, 의, 학습, 능력, 추론, 능력, 지각, 능..."
1,하나의 인프라 기술이기도 하다,"[하나, 의, 인프라, 기술, 이, 기, 도, 하, 다]"
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...,"[인간, 을, 포함, 한, 동물, 이, 갖, 고, 있, 는, 지능, 즉, natur..."
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...,"[지능, 을, 갖, 고, 있, 는, 기능, 을, 갖춘, , 컴퓨터, 시스템, 이,..."
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다,"[일반, 적, 으로, 범용, 컴퓨터, 에, 적용, 한다고, 가정, 한다]"
5,이 용어는 또한 그와 같은 지능을 만들 수 있는 방법론이나 실현 가능성 등을 연구하...,"[이, 용어, 는, 또한, 그, 와, 같, 은, 지능, 을, 만들, 수, 있, 는,..."


In [3]:
words = set()
data['morphs'].map(lambda x: words.update(set(x)))

word_to_index = {w: i + 4 for i, w in enumerate(words)}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1
word_to_index["SOS"] = 2
word_to_index["EOS"] = 3

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [4]:
chars = set([w_i for w in words for w_i in w])
chars = sorted(list(chars))

char_to_index = {c: i + 2 for i, c in enumerate(chars)}
char_to_index["PAD"] = 0
char_to_index["UNK"] = 1

index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [5]:
max_len_char = 10
max_len_word = 50

def integer_coding(sentences, max_len_char, max_len_word):
    word_data = []
    
    for i, word in enumerate(sentences):
        word_data.append(word_to_index[word])
        char_indice = [[char_to_index[char] for char in word]]
        char_indice = pad_sequences(char_indice, maxlen=max_len_char, padding='post', value=0)

        for chars_of_token in char_indice:
            if len(chars_of_token) > max_len_char:
                continue
        
        if i == 0:
            char_data = char_indice
        else:
            char_data = np.vstack([char_data, char_indice])
    
    pad_char_data = max_len_word - char_data.shape[0]
    
    char_data = np.pad(char_data, ((0, pad_char_data), (0, 0)), 'constant', constant_values=0)
    word_data = pad_sequences([word_data], maxlen=max_len_word, padding='post', value=0)
    
    return word_data, char_data

In [6]:
data.loc[0, "morphs"]

['인공지능',
 '또는',
 'AI',
 '는',
 '인간',
 '의',
 '학습',
 '능력',
 '추론',
 '능력',
 '지각',
 '능력',
 '그',
 '외',
 '에',
 '인공',
 '적',
 '으로',
 '구현',
 '한',
 '컴퓨터',
 '프로그램',
 '또는',
 '이',
 '를',
 '포함',
 '한',
 '컴퓨터',
 '시스템',
 '이',
 '다']

In [7]:
X_word_data, X_char_data = integer_coding(data.loc[3, "morphs"], max_len_char, max_len_word)
X_char_data

array([[79, 32,  0,  0,  0,  0,  0,  0,  0,  0],
       [70,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [18,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [23,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [75,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [31,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [28, 32,  0,  0,  0,  0,  0,  0,  0,  0],
       [70,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [18, 81,  0,  0,  0,  0,  0,  0,  0,  0],
       [14,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [83, 87, 84,  0,  0,  0,  0,  0,  0,  0],
       [59, 57, 85,  0,  0,  0,  0,  0,  0,  0],
       [72,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [47,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [73, 17,  0,  0,  0,  0,  0,  0,  0,  0],
       [71,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [79, 32,  0,  0,  0,  0,  0,  0,  0,  0],
       [70,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [28, 22,  0,  0,  0,  0,  0,  0,  0,  0],
       [37,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [63,  0,  0, 

In [8]:
X_word_data

array([[13, 60, 68, 42, 38,  7, 71, 60, 63, 11, 40,  6, 72, 12, 29, 19,
        13, 60, 24, 73, 14,  4, 48, 41, 15, 46, 59, 72, 55,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]], dtype=int32)

# 국민청원 데이터셋

In [9]:
from collcetions import Counter

df = pd.read_table('data/test_data_cleaned.txt', header=None, names=['text']).dropna()

In [10]:
df = pd.read_table('data/test_data_cleaned.txt', header=None, names=['text']).dropna()
tokenizer = konlpy.tag.Mecab()

df['text'] = df['text'].str.replace(",", "")
df['morphs'] = df['text'].map(lambda x: tokenizer.morphs(x))
df

,text,morphs
0,국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 ...,"[국민, 과, 소통, 하, 시, 고, 자유, 롭, 고, 행복, 한, 나라, 를, 만..."
1,기해년 새해 복 많이 받으십시오,"[기해, 년, 새해, 복, 많이, 받, 으십시오]"
2,저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고...,"[저, 는, 경북, 울진군, 북면, 부, 구검, 성, 로, 12, 번지, 에, 살,..."
3,저는 8기의 원전이 가동 건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 ...,"[저, 는, 8, 기, 의, 원전, 이, 가동, 건설, 되, 고, 있, 는, 이곳,..."
4,간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다,"[간절, 한, 마음, 을, 담, 아, 대통령, 께, 다음, 과, 같이, 호소, 드립니다]"
...,...,...
214943,알바가 검사 안한 잘못도 있지만 미성년자가 술을 마시러 온거 자체가 잘못되었는데 업...,"[알바, 가, 검사, 안, 한, 잘못, 도, 있, 지만, 미성년자, 가, 술, 을,..."
214944,그러므로 앞으로 미성년자의 음주가 적발될시 미성년들도 강한 처벌을 받아야하게 바꾸거...,"[그러므로, 앞, 으로, 미성년자, 의, 음주, 가, 적발, 될, 시, 미성년, 들..."
214946,정권이 바뀐다 해도 세금이 계속 증가만 하고 내려가지는 않는데요! 어디에 어떻게 써...,"[정권, 이, 바뀐다, 해도, 세금, 이, 계속, 증가, 만, 하, 고, 내려, 가..."
214947,그리고 예산을 받아서 제대로 쓰여졌는지 감사하는 기구는 어디였고 어떠한 이유로 지연...,"[그리고, 예산, 을, 받, 아서, 제대로, 쓰여졌, 는지, 감사, 하, 는, 기구..."


In [48]:
min_freq = 5

# words = set()
# df['morphs'].map(lambda x: words.update(set(x)))
ct = Counter([char for line in df.morphs.values for char in line ])
thresholded_word_vocab = [ key for key, value in ct.items() if value >= min_freq]

word_to_index = {w: i + 2 for i, w in enumerate(thresholded_word_vocab)}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [70]:
# chars = set([w_i for w in words for w_i in w])
chars = set([w_i for w in thresholded_word_vocab for w_i in w])
chars = sorted(list(chars))

char_to_index = {c: i + 2 for i, c in enumerate(chars)}
char_to_index["PAD"] = 0
char_to_index["UNK"] = 1

thresholded_char_vocab = list(char_to_index.keys())

index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [71]:
import pickle

word_vocab = [index_to_word, word_to_index]
char_vocab = [index_to_char, char_to_index]

with open('data/word_vocabulary.p', 'wb') as f:
    pickle.dump(word_vocab, f)

with open('data/char_vocabulary.p', 'wb')as f:
    pickle.dump(char_vocab, f)

In [82]:
max_len_char = 6
max_len_word = 25

def integer_coding(sentences, max_len_char, max_len_word):
    word_data = []
    
    for i, word in enumerate(sentences):
        
        if word in thresholded_vocab:
            word_data.append(word_to_index[word])
        elif word not in thresholded_word_vocab:
            word_data.append(word_to_index["UNK"])
            
        char_indice = [[char_to_index[char] if char in thresholded_char_vocab else char_to_index["UNK"] for char in word]]
        char_indice = pad_sequences(char_indice, maxlen=max_len_char, padding='post', value=0)

        for chars_of_token in char_indice:
            if len(chars_of_token) > max_len_char:
                continue
        
        if i == 0:
            char_data = char_indice
        else:
            char_data = np.vstack([char_data, char_indice])
    
    pad_char_data = max_len_word - char_data.shape[0]
    
    if pad_char_data > 0:
        char_data = np.pad(char_data, ((0, pad_char_data), (0, 0)), 'constant', constant_values=0)
    else:
        char_data = char_data[:max_len_word]
        
    word_data = pad_sequences([word_data], maxlen=max_len_word, padding='post', value=0)
    
    return np.expand_dims(word_data, 0), np.expand_dims(char_data, 0)

In [83]:
sample_df = df.loc[:100, :]
sample_df

,text,morphs
0,국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 ...,"[국민, 과, 소통, 하, 시, 고, 자유, 롭, 고, 행복, 한, 나라, 를, 만..."
1,기해년 새해 복 많이 받으십시오,"[기해, 년, 새해, 복, 많이, 받, 으십시오]"
2,저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고...,"[저, 는, 경북, 울진군, 북면, 부, 구검, 성, 로, 12, 번지, 에, 살,..."
3,저는 8기의 원전이 가동 건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 ...,"[저, 는, 8, 기, 의, 원전, 이, 가동, 건설, 되, 고, 있, 는, 이곳,..."
4,간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다,"[간절, 한, 마음, 을, 담, 아, 대통령, 께, 다음, 과, 같이, 호소, 드립니다]"
...,...,...
95,앞으로 제2의 저의 자녀와 같은 장애인 취업후 사직을 강요하는 것을 바로잡고자 청원...,"[앞, 으로, 제, 2, 의, 저, 의, 자녀, 와, 같, 은, 장애, 인, 취업,..."
96,저의 자녀는 1990년생이며 2005년 교통사고 후 후유증으로 2010년 6월 9일...,"[저, 의, 자녀, 는, 1990, 년, 생, 이, 며, 2005, 년, 교통사고,..."
97,아울러 고용노동부전주지청 노동위원회에 해고무효소송을 2018년 12월 31일 했습니다,"[아울러, 고용, 노동부, 전주, 지청, 노동, 위원회, 에, 해고, 무효, 소송,..."
98,두번다시 사회적약자인 장애인채용 후 사직이라는 일이 발생되지 않도록 국민여러분의 큰...,"[두, 번, 다시, 사회, 적, 약자, 인, 장애, 인, 채용, 후, 사직, 이, ..."


In [84]:
from tqdm.notebook import tqdm

word_array = np.array([])
char_array = np.array([])

tot = sample_df.shape[0]

for i, t in tqdm(sample_df.iterrows(), total=tot):
    word_vector, char_vector = integer_coding(t.morphs, max_len_char, max_len_word)

    if i == 0:
        word_array = word_vector
        char_array = char_vector
    else:
        word_array = np.append(word_array, word_vector, axis=0)
        char_array = np.append(char_array, char_vector, axis=0)
        
word_array.shape

  0%|          | 0/95 [00:00<?, ?it/s]

(95, 1, 25)

In [85]:
char_array.shape

(95, 25, 6)

In [86]:
char_array[0]

array([[ 170,  678,    0,    0,    0,    0],
       [ 157,    0,    0,    0,    0,    0],
       [ 854, 1376,    0,    0,    0,    0],
       [1465,    0,    0,    0,    0,    0],
       [ 895,    0,    0,    0,    0,    0],
       [ 146,    0,    0,    0,    0,    0],
       [1092, 1063,    0,    0,    0,    0],
       [ 562,    0,    0,    0,    0,    0],
       [ 146,    0,    0,    0,    0,    0],
       [1479,  730,    0,    0,    0,    0],
       [1467,    0,    0,    0,    0,    0],
       [ 276,  510,    0,    0,    0,    0],
       [ 586,    0,    0,    0,    0,    0],
       [ 602,  442,    0,    0,    0,    0],
       [ 199,    0,    0,    0,    0,    0],
       [1059, 1473,    0,    0,    0,    0],
       [1553,  937,    0,    0,    0,    0],
       [ 146,    0,    0,    0,    0,    0],
       [ 145,  897,    0,    0,    0,    0],
       [ 377, 1376,  556,    0,    0,    0],
       [ 227,    0,    0,    0,    0,    0],
       [1133,  143,    0,    0,    0,    0],
       [ 1

In [87]:
from tqdm.notebook import tqdm

word_array = np.array([])
char_array = np.array([])

tot = df.shape[0]

for i, t in tqdm(df.iterrows(), total=tot):
    word_vector, char_vector = integer_coding(t.morphs, max_len_char, max_len_word)

    if i == 0:
        word_array = word_vector
        char_array = char_vector
    else:
        word_array = np.append(word_array, word_vector, axis=0)
        char_array = np.append(char_array, char_vector, axis=0)

  0%|          | 0/172562 [00:00<?, ?it/s]

In [88]:
np.save("data/word_array.npy", word_array)
np.save("data/char_array.npy", char_array)

In [89]:
X_word_data, X_char_data = integer_coding(df.loc[0, "morphs"], max_len_char, max_len_word)
X_char_data

array([[[ 170,  678,    0,    0,    0,    0],
        [ 157,    0,    0,    0,    0,    0],
        [ 854, 1376,    0,    0,    0,    0],
        [1465,    0,    0,    0,    0,    0],
        [ 895,    0,    0,    0,    0,    0],
        [ 146,    0,    0,    0,    0,    0],
        [1092, 1063,    0,    0,    0,    0],
        [ 562,    0,    0,    0,    0,    0],
        [ 146,    0,    0,    0,    0,    0],
        [1479,  730,    0,    0,    0,    0],
        [1467,    0,    0,    0,    0,    0],
        [ 276,  510,    0,    0,    0,    0],
        [ 586,    0,    0,    0,    0,    0],
        [ 602,  442,    0,    0,    0,    0],
        [ 199,    0,    0,    0,    0,    0],
        [1059, 1473,    0,    0,    0,    0],
        [1553,  937,    0,    0,    0,    0],
        [ 146,    0,    0,    0,    0,    0],
        [ 145,  897,    0,    0,    0,    0],
        [ 377, 1376,  556,    0,    0,    0],
        [ 227,    0,    0,    0,    0,    0],
        [1133,  143,    0,    0,  

In [90]:
X_word_data

array([[[ 3,  4,  5,  6,  7,  8,  9,  7, 10, 11, 12, 13, 14, 15, 16, 17,
          7, 18, 19, 20, 21,  3, 22, 13, 23]]], dtype=int32)